In [ ]:
import warnings
warnings.filterwarnings("ignore")
from copy import deepcopy
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd

from tqdm import tqdm
import torch
device = torch.device('cpu')


import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

from sklearn.utils import check_random_state

# implementing OPE of the IPWLearner using synthetic bandit data
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

from scipy.special import softmax
from abc import ABCMeta


from obp.ope import (
    RegressionModel,
    DirectMethod as DM,
)

from my_utils import (
    eval_policy,
    generate_dataset,
    create_simluation_data_from_pi,
    get_train_data,
    CFModel,
    CustomCFDataset,
    NeighborhoodModel,
    BPRModel
)
random_state=12345
random_ = check_random_state(random_state)

## `calc_reward` Function
Calculates the expected reward of a policy by computing the weighted average of true reward probabilities.

### Parameters
- `dataset` (dict): Contains dataset information including `q_x_a`, the true reward probabilities for each user-action pair
- `policy` (numpy.ndarray): Policy probabilities with shape [n_users, n_actions, 1]

### Returns
- `numpy.ndarray`: A single-element array containing the expected policy reward

### Mathematical Formulation
Implements: $R_{gt} = \frac{1}{n}\sum_{i=1}^{n}\sum_{j=1}^{m}{\pi_{ij} \cdot p_{ij}}$

Where:
- $\pi_{ij}$ is the policy probability for user $i$ choosing action $j$
- $p_{ij}$ is the true reward probability for user $i$ choosing action $j$ (stored in `q_x_a`)

In [2]:
def calc_reward(dataset, policy):
    return np.array([np.sum(dataset['q_x_a'] * policy.squeeze(), axis=1).mean()])

In [3]:
pd.options.display.float_format = '{:,.4f}'.format

## `IPWPolicyLoss` Class

This class implements an Inverse Propensity Weighting (IPW) loss function for counterfactual policy learning from offline bandit data.

### Mathematical Formulation
The loss implements the IPW estimator as a differentiable function:

$$\mathcal{L}_{IPW} = \frac{1}{n}\sum_{i=1}^{n} \frac{\pi_e(a_i|x_i)}{\pi_0(a_i|x_i)} \cdot r_i \cdot \log(\pi_e(a_i|x_i))$$

Where:
- $\pi_e(a_i|x_i)$ is the probability of the new policy taking action $a_i$ for context $x_i$
- $\pi_0(a_i|x_i)$ is the propensity score (probability of the logging policy)
- $r_i$ is the observed reward
- $n$ is the batch size

### Parameters
- **log_eps** (float): Small constant added to prevent numerical instability in log calculations

### Method: `forward`
- **pscore** (Tensor): Propensity scores from original logging policy
- **scores** (Tensor): Model-estimated reward predictions for each action (not being used)
- **policy_prob** (Tensor): Probabilities from current policy being optimized
- **original_policy_rewards** (Tensor): Observed rewards from logged data
- **original_policy_actions** (Tensor): Actions that were taken in the logged data

### Implementation Notes
- Importance weights (`iw`) are detached from the computation graph
- Uses the REINFORCE policy gradient method
- The implementation includes commented-out code for more advanced variants

In [4]:
class IPWPolicyLoss(nn.Module):
    def __init__(self, log_eps=1e-10):
        super(IPWPolicyLoss, self).__init__()
        self.log_eps = log_eps

    def forward(self, pscore, scores, policy_prob, original_policy_rewards, original_policy_actions):
        n = original_policy_actions.shape[0]

        pi_e_at_position = policy_prob[torch.arange(n), original_policy_actions].squeeze()
        iw = pi_e_at_position / pscore
        iw = iw.detach()
        # q_hat_at_position = scores[torch.arange(n), original_policy_actions].squeeze()
        # dm_grads = (scores * policy_prob.detach() * torch.log(policy_prob)).sum(dim=1)
        log_pi = torch.log(pi_e_at_position).squeeze()
        
        # reinforce trick step
        # reinforce_grad = ((iw * (original_policy_rewards - q_hat_at_position) * log_pi) / iw.sum()) + dm_grads
        reinforce_grad = iw * original_policy_rewards * log_pi
        
        return reinforce_grad.mean()

## `SNDRPolicyLoss` Class

This class implements a Self-Normalized Doubly Robust (SNDR) loss function for counterfactual policy learning from offline bandit data.

### Mathematical Formulation
The loss combines IPW with direct method estimates for variance reduction:

$$\mathcal{L}_{SNDR} = \frac{1}{n}\sum_{i=1}^{n} \left( \frac{\sum_{i=1}^{n}\frac{\pi_e(a_i|x_i)}{\pi_0(a_i|x_i)} \cdot (r_i - \hat{q}(x_i,a_i))}{\sum_{i=1}^{n}\frac{\pi_e(a_i|x_i)}{\pi_0(a_i|x_i)}} + \sum_{a}\pi_e(a|x_i)\hat{q}(x_i,a) \right) \cdot \log(\pi_e(a_i|x_i))$$

Where:
- $\pi_e(a_i|x_i)$ is the probability from the new policy
- $\pi_0(a_i|x_i)$ is the propensity score from the logging policy
- $r_i$ is the observed reward
- $\hat{q}(x_i,a_i)$ is the estimated reward from a direct model
- $n$ is the batch size

### Parameters
- **log_eps** (float): Small constant added to prevent numerical instability in log calculations

### Method: `forward`
- **pscore** (Tensor): Propensity scores from original logging policy
- **scores** (Tensor): Model-estimated reward predictions for each action
- **policy_prob** (Tensor): Probabilities from current policy being optimized
- **original_policy_rewards** (Tensor): Observed rewards from logged data
- **original_policy_actions** (Tensor): Actions that were taken in the logged data

### Implementation Notes
- Combines direct method rewards with importance-weighted corrections
- Self-normalizes the importance weights by dividing by their sum
- Generally provides lower variance estimates than pure IPW approaches

In [5]:
class SNDRPolicyLoss(nn.Module):
    def __init__(self, log_eps=1e-10):
        super(SNDRPolicyLoss, self).__init__()
        self.log_eps = log_eps

    def forward(self, pscore, scores, policy_prob, original_policy_rewards, original_policy_actions):
        n = original_policy_actions.shape[0]

        pi_e_at_position = policy_prob[torch.arange(n), original_policy_actions].squeeze()
        iw = pi_e_at_position / pscore
        iw = iw.detach()
        q_hat_at_position = scores[torch.arange(n), original_policy_actions].squeeze()
        dm_reward = (scores * policy_prob.detach()).sum(dim=1)
        log_pi = torch.log(pi_e_at_position).squeeze()
        
        # reinforce trick step
        r_hat = ((iw * (original_policy_rewards - q_hat_at_position)) / iw.sum()) + dm_reward
        reinforce_grad = r_hat * log_pi
        return reinforce_grad.mean()

## `get_opl_results_dict` Function

This function processes evaluation results from various offline policy learning (OPL) estimators and computes summary statistics.

### Parameters
- **reg_results** (numpy.ndarray): Results from regression-based direct method estimator
- **conv_results** (numpy.ndarray): Results from various estimators including true rewards and embeddings quality metrics

### Returns
- **dict**: A dictionary containing the following metrics:
  - `policy_rewards`: Mean true reward of the learned policy
  - Error metrics (absolute difference between estimator and true reward):
    - `ipw`: Inverse Propensity Weighting estimator error
    - `reg_dm`: Regression-based Direct Method estimator error
    - `conv_dm`: Convolution-based Direct Method estimator error
    - `conv_dr`: Convolution-based Doubly Robust estimator error
    - `conv_sndr`: Convolution-based Self-Normalized Doubly Robust estimator error
  - Variance metrics for each estimator:
    - `ipw_var`, `reg_dm_var`, `conv_dm_var`, `conv_dr_var`, `conv_sndr_var`
  - Embedding quality metrics:
    - `action_diff_to_real`: RMSE between learned and real action embeddings
    - `action_delta`: RMSE between learned and original action embeddings
    - `context_diff_to_real`: RMSE between learned and real context embeddings
    - `context_delta`: RMSE between learned and original context embeddings

### Implementation Notes
- Uses the first column of `conv_results` as the ground truth reward
- Contains commented-out code for percentage error calculations
- Computes absolute errors rather than signed differences

In [6]:
def get_opl_results_dict(reg_results, conv_results):
    reward = conv_results[:, 0]
    return    dict(
                policy_rewards=np.mean(reward),
                ipw=np.mean(abs(conv_results[: ,3] - reward)),
                reg_dm=np.mean(abs(reg_results - reward)),
                conv_dm=np.mean(abs(conv_results[: ,1] - reward)),
                conv_dr=np.mean(abs(conv_results[: ,2] - reward)),
                conv_sndr=np.mean(abs(conv_results[: ,4] - reward)),

                ipw_var=np.var(conv_results[: ,3]),
                reg_dm_var=np.var(reg_results),
                conv_dm_var=np.var(conv_results[: ,1]),
                conv_dr_var=np.var(conv_results[: ,2]),
                conv_sndr_var=np.var(conv_results[: ,4]),

                                
                # ipw_p_err=np.mean(abs(conv_results[: ,3] - reward) / reward) * 100,
                # reg_dm_p_err=np.mean(abs(reg_results - reward) / reward) * 100,
                # conv_dm_p_err=np.mean(abs(conv_results[: ,1] - reward) / reward) * 100,
                # conv_dr_p_err=np.mean(abs(conv_results[: ,2] - reward) / reward) * 100,
                # conv_sndr_p_err=np.mean(abs(conv_results[: ,4] - reward) / reward) * 100,
                
                action_diff_to_real=np.mean(conv_results[: ,5]),
                action_delta=np.mean(conv_results[: ,6]),
                context_diff_to_real=np.mean(conv_results[: ,7]),
                context_delta=np.mean(conv_results[: ,8])
                )

## `train` Function

This function trains a policy model with Self-Normalized Doubly Robust (SNDR) loss for counterfactual policy learning.

### Parameters
- **model** (CFModel): The policy model to be trained, which maps users to action probabilities
- **train_loader** (DataLoader): PyTorch data loader containing training data with user indices, actions, rewards, and logging policy probabilities
- **neighborhood_model** (NeighborhoodModel): Model that provides reward estimates based on neighborhood information
- **num_epochs** (int, default=1): Number of training epochs
- **lr** (float, default=0.0001): Learning rate for the Adam optimizer
- **device** (str or torch.device, default='cpu'): Device to run the training on

### Process Flow
1. Initializes an Adam optimizer and SNDR loss criterion
2. For each epoch:
   - Iterates through batches from the data loader
   - Moves data to specified device (CPU/GPU)
   - Gets policy probabilities by running the model on user indices
   - Computes propensity scores from the logging policy
   - Gets reward predictions from neighborhood model
   - Calculates loss using the SNDR criterion
   - Performs backpropagation and optimization
   - Tracks and displays running loss statistics

### Implementation Notes
- Uses `tqdm` for progress visualization
- Contains commented-out code for neighborhood model updates

In [7]:
# 4. Define the training function
def train(model, train_loader, neighborhood_model, num_epochs=1, lr=0.0001, device='cpu'):

    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr) # here we can change the learning rate
    criterion = SNDRPolicyLoss()

    model.train() # Set the model to training mode
    tq = tqdm(range(num_epochs))
    for epoch in tq:
        running_loss = 0.0
        total_samples = 0
        
        for user_idx, action_idx, rewards, original_prob in train_loader:
            # Move data to GPU if available
            if torch.cuda.is_available():
                user_idx = user_idx.to(device) 
                action_idx = action_idx.to(device)
                rewards = rewards.to(device)
                original_prob = original_prob.to(device) 
            
            # Forward pass
            policy = model(user_idx)
            pscore = original_prob[torch.arange(user_idx.shape[0]), action_idx.type(torch.long)]
            
            scores = torch.tensor(neighborhood_model.predict(user_idx.cpu().numpy()), device='cpu')
            
            loss = criterion(
                              pscore,
                              scores,
                              policy, 
                              rewards, 
                              action_idx.type(torch.long), 
                              )
            
            # Zero the gradients Backward pass and optimization
            optimizer.zero_grad()

            loss.backward()                        
            optimizer.step()
            
            # update neighborhood
            # action_emb, context_emb = model.get_params()
            
            # Calculate running loss and accuracy
            running_loss += loss.item()
            total_samples += 1

            # Print statistics after each epoch
            epoch_loss = running_loss / total_samples
            tq.set_description(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
            
        # neighborhood_model.update(action_emb.detach().numpy(), context_emb.detach().numpy())


## `trainer_trial` Function

This function runs policy learning experiments using offline bandit data and evaluates various estimators.

### Parameters
- **num_runs** (int): Number of experimental runs per training size
- **num_neighbors** (int): Number of neighbors to consider in the neighborhood model
- **num_rounds_list** (list): List of training set sizes to evaluate
- **dataset** (dict): Contains dataset information including embeddings, action probabilities, and reward probabilities
- **batch_size** (int): Batch size for training the policy model
- **num_epochs** (int): Number of training epochs for each experiment
- **lr** (float, default=0.001): Learning rate for the optimizer

### Process Flow
1. Initializes result structures and retrieval models
2. For each training size in `num_rounds_list`:
   - Creates a uniform logging policy and simulates data
   - Generates training data for offline learning
   - Fits regression and neighborhood models for reward estimation
   - Initializes and trains a counterfactual policy model
   - Evaluates policy performance using various estimators
   - Collects metrics on policy reward and embedding quality

### Returns
- **DataFrame**: Results table with rows indexed by training size and columns for various metrics:
  - `policy_rewards`: True expected reward of the learned policy
  - Various estimator errors (`ipw`, `reg_dm`, `conv_dm`, `conv_dr`, `conv_sndr`)
  - Variance metrics for each estimator
  - Embedding quality metrics comparing learned representations to ground truth

### Implementation Notes
- Uses uniform random logging policy for collecting offline data
- Employs Self-Normalized Doubly Robust (SNDR) policy learning
- Measures embedding quality via RMSE to original/ground truth embeddings

In [8]:
def trainer_trial(
                  num_runs,
                  num_neighbors,
                  num_rounds_list,
                  dataset,
                  batch_size,
                  num_epochs,
                  lr=0.001
                  ):
    # Define device at the beginning
    device = torch.device('cpu')  # Force CPU usage
    
    dm = DM()
    results = {}

    our_x, our_a = dataset["our_x"], dataset["our_a"]
    emb_x, emb_a = dataset["emb_x"], dataset["emb_a"]
    
    original_x, original_a = dataset["original_x"], dataset["original_a"]
    n_users, n_actions, emb_dim = dataset["n_users"], dataset["n_actions"], dataset["emb_dim"]
    first = True
    zero = True
    for train_size in num_rounds_list:
        reg_results, conv_results = [], []
        for run in range(num_runs):

            pi_0 = np.ones_like(dataset["q_x_a"])/(dataset["n_actions"])
            original_policy_prob = np.expand_dims(pi_0, -1)
            simulation_data = create_simluation_data_from_pi(
                                                            pi_0,
                                                            dataset["q_x_a"],
                                                            dataset["n_users"],
                                                            dataset["n_actions"],
                                                            random_state=train_size*(run+1)
                                                            )
            
            # test_data = get_test_data(dataset, simulation_data, n_test_data)
            
            # idx = np.arange(train_size) + n_test_data
            idx = np.arange(train_size)
            train_data = get_train_data(n_actions, train_size, simulation_data, idx, our_x)
            
            regression_model = RegressionModel(
                                                n_actions=n_actions,
                                                action_context=our_x,
                                                base_model=LogisticRegression(random_state=12345)
                                                )
            
            regression_model.fit(train_data['x'], 
                        train_data['a'],
                        train_data['r'],
                        original_policy_prob[train_data['x_idx'],
                        train_data['a']].squeeze()
                        )

            neighberhoodmodel = NeighborhoodModel(
                                                    train_data['x_idx'],
                                                    train_data['a'], 
                                                    our_a,
                                                    our_x, 
                                                    train_data['r'], 
                                                    num_neighbors=num_neighbors
                                                )
            

            model = CFModel(
                            n_users, 
                            n_actions, 
                            emb_dim, 
                            initial_user_embeddings=torch.tensor(our_x, device='cpu'), 
                            initial_actions_embeddings=torch.tensor(our_a, device='cpu')
                            )
            
            cf_dataset =  CustomCFDataset(
                                       train_data['x_idx'], 
                                       train_data['a'], 
                                       train_data['r'], 
                                       original_policy_prob[train_data['x_idx']]
                                       )
            
            train_loader = DataLoader(cf_dataset, batch_size=batch_size, shuffle=False)
            if first:
                policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)
                conv_results.append(eval_policy(neighberhoodmodel, train_data, original_policy_prob[train_data['x_idx']], policy))
                conv_results[-1] = np.append(calc_reward(dataset, policy), conv_results[-1])
                conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_a-our_a)**2)), np.sqrt(np.mean((original_a-our_a)**2))])
                conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_x-our_x)**2)), np.sqrt(np.mean((original_x-our_x)**2))])
                reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], regression_model.predict(train_data['x']))
                reg_results.append(reg_dm)
                first = False
                reg_results = np.array(reg_results)
                conv_results = np.array(conv_results)
                results[0] = get_opl_results_dict(reg_results, conv_results)
                reg_results, conv_results = [], []
            
            train(model, train_loader, neighberhoodmodel, num_epochs=num_epochs, lr=lr, device='cpu')
            # neighborhood_model.update(model.get_params()[0].detach().numpy(), model.get_params()[1].detach().numpy())'

            our_x, our_a = model.get_params()
            our_a, our_x = our_a.detach().cpu().numpy(), our_x.detach().cpu().numpy()

            policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)

            # reg_dm = dm.estimate_policy_value(policy[test_data['x_idx']], regression_model.predict(test_data['x']))
            reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], regression_model.predict(train_data['x']))

            reg_results.append(reg_dm)

            # conv_results.append(eval_policy(neighberhoodmodel, test_data, original_policy_prob[test_data['x_idx']], policy))
            conv_results.append(eval_policy(neighberhoodmodel, train_data, original_policy_prob[train_data['x_idx']], policy))

            conv_results[-1] = np.append(calc_reward(dataset, policy), conv_results[-1])
            conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_a-our_a)**2)), np.sqrt(np.mean((original_a-our_a)**2))])

            # temp.append(np.mean((emb_a-our_a)**2, axis=0))

            conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_x-our_x)**2)), np.sqrt(np.mean((original_x-our_x)**2))])
            
            our_a, our_x = original_a.copy(), original_x.copy()

        reg_results = np.array(reg_results)
        conv_results = np.array(conv_results)

        results[train_size] = get_opl_results_dict(reg_results, conv_results)
    
    return pd.DataFrame.from_dict(results, orient='index')

## Learning

We will run several simulations on a generated dataset, the dataset is generated like this:
$$ \text{We have users U and actions A } u_i \sim N(0, I_{emb_dim}) \ a_i \sim N(0, I_{emb_dim})$$
$$ p_{ij} = 1 / (5 + e^{-(u_i.T a_j)}) $$
$$r_{ij} \sim Bin(p_{ij})$$

We have a policy $\pi$
and it's ground truth reward is calculated by
$$R_{gt} = \sum_{i}{\sum_{j}{\pi_{ij} * p_{ij}}} $$

Our parameters for the dataset will be
$$EmbDim = 5$$
$$NumActions= 150$$
$$NumUsers = 150$$
$$NeighborhoodSize = 6$$

to learn a new policy from $\pi$ we will sample from:
$$\pi_{start} = (1-\epsilon)*\pi + \epsilon * \pi_{random}$$

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [10]:
num_runs = 1

In [11]:
dataset_params = dict(
                    n_actions= 150,
                    n_users = 150,
                    emb_dim = 5,
                    # sigma = 0.1,
                    eps = 0.4 # this is the epsilon for the noise in the ground truth policy representation
                    )

train_dataset = generate_dataset(dataset_params)

In [12]:
num_runs = 1
batch_size = 50
num_neighbors = 6
num_rounds_list = [1, 2, 3, 4, 5, 10, 20]

### 1

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.005$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [13]:
df4 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, num_epochs=1, lr=0.005)

Epoch [1/1], Loss: -1.0260: 100%|██████████| 1/1 [00:00<00:00,  3.39it/s]


In [14]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.1745,0.0063,0.0461,0.0032,0.0472,0.0493,0.4515,0.0000,0.7152,0.0000
1,0.1747,0.0052,0.0463,0.0032,0.0455,0.0493,0.4490,0.0118,0.7160,0.0076
2,0.1748,0.0206,0.0152,0.0113,0.0148,0.0153,0.4468,0.0188,0.7162,0.0129
3,0.1750,0.0126,0.0307,0.0058,0.0250,0.0273,0.4451,0.0244,0.7159,0.0174
4,0.1752,0.0228,0.0335,0.0208,0.0380,0.0396,0.4414,0.0296,0.7166,0.0209
5,0.1753,0.0192,0.0195,0.0044,0.0182,0.0183,0.4413,0.0338,0.7174,0.0234
10,0.1763,0.0113,0.0247,0.0096,0.0028,0.0043,0.4275,0.0571,0.7203,0.0414
20,0.1785,0.0085,0.0336,0.0037,0.0085,0.0091,0.4026,0.1055,0.7308,0.0820


### 2

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.001$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [15]:
df5 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, num_epochs=1, lr=0.001)

Epoch [1/1], Loss: -0.9791: 100%|██████████| 1/1 [00:00<00:00,  3.15it/s]


In [16]:
df5

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.1745,0.0063,0.0461,0.0032,0.0472,0.0493,0.0000,0.0000,0.0000,0.0000,0.0000,0.4515,0.0000,0.7152,0.0000
1,0.1746,0.0061,0.0462,0.0032,0.0468,0.0493,0.0000,0.0000,0.0000,0.0000,0.0000,0.4510,0.0024,0.7153,0.0015
2,0.1746,0.0196,0.0150,0.0111,0.0136,0.0138,0.0000,0.0000,0.0000,0.0000,0.0000,0.4505,0.0038,0.7154,0.0026
3,0.1746,0.0153,0.0302,0.0062,0.0275,0.0291,0.0000,0.0000,0.0000,0.0000,0.0000,0.4501,0.0049,0.7153,0.0035
4,0.1747,0.0223,0.0329,0.0210,0.0369,0.0377,0.0000,0.0000,0.0000,0.0000,0.0000,0.4494,0.0059,0.7154,0.0042
5,0.1747,0.0174,0.0188,0.0055,0.0172,0.0165,0.0000,0.0000,0.0000,0.0000,0.0000,0.4493,0.0067,0.7155,0.0046
10,0.1749,0.0133,0.0233,0.0064,0.0077,0.0086,0.0000,0.0000,0.0000,0.0000,0.0000,0.4463,0.0111,0.7158,0.0081
20,0.1753,0.0072,0.0303,0.0003,0.0056,0.0060,0.0000,0.0000,0.0000,0.0000,0.0000,0.4406,0.0194,0.7165,0.0149


### 3

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.003$$
$$n_{epochs} = 10$$
$$BatchSize=50$$

In [17]:
df6 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, num_epochs=10, lr=0.003)

Epoch [10/10], Loss: -5.7527: 100%|██████████| 10/10 [00:03<00:00,  3.21it/s]


In [18]:
df6

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.1745,0.0063,0.0461,0.0032,0.0472,0.0493,0.0000,0.0000,0.0000,0.0000,0.0000,0.4515,0.0000,0.7152,0.0000
1,0.1755,0.0050,0.0475,0.0040,0.0377,0.0533,0.0000,0.0000,0.0000,0.0000,0.0000,0.4401,0.0678,0.7224,0.0508
2,0.1766,0.0139,0.0170,0.0114,0.0119,0.0123,0.0000,0.0000,0.0000,0.0000,0.0000,0.4286,0.1085,0.7290,0.0794
3,0.1780,0.0028,0.0348,0.0057,0.0165,0.0226,0.0000,0.0000,0.0000,0.0000,0.0000,0.4179,0.1427,0.7323,0.1080
4,0.1788,0.0400,0.0385,0.0174,0.0608,0.0786,0.0000,0.0000,0.0000,0.0000,0.0000,0.4063,0.1796,0.7543,0.1393
5,0.1792,0.0096,0.0236,0.0070,0.0076,0.0078,0.0000,0.0000,0.0000,0.0000,0.0000,0.4126,0.2120,0.7751,0.1660
10,0.1777,0.0641,0.0235,0.0368,0.0624,0.0753,0.0000,0.0000,0.0000,0.0000,0.0000,0.4610,0.3969,0.9609,0.4009
20,0.1643,0.1125,0.0119,0.0057,0.1439,0.1490,0.0000,0.0000,0.0000,0.0000,0.0000,0.8490,0.8539,1.6881,1.1560


### 4

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.05$$
$$n_{epochs} = 10$$
$$BatchSize=150$$

In [19]:
df7 = trainer_trial(num_runs, num_neighbors, num_rounds_list[:-3], train_dataset, batch_size+100, num_epochs=10, lr=0.05)

Epoch [10/10], Loss: -6.0559: 100%|██████████| 10/10 [00:00<00:00, 23.12it/s]


In [20]:
df7

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.1745,0.0063,0.0461,0.0032,0.0472,0.0493,0.0000,0.0000,0.0000,0.0000,0.0000,0.4515,0.0000,0.7152,0.0000
1,0.1802,0.0186,0.0534,0.0084,0.0120,0.1022,0.0000,0.0000,0.0000,0.0000,0.0000,0.5694,0.4909,0.9169,0.4731
2,0.1831,0.1712,0.0240,0.0094,0.0062,0.1386,0.0000,0.0000,0.0000,0.0000,0.0000,0.7891,0.8231,1.2189,0.8180
3,0.1851,0.1271,0.0410,0.0033,0.0063,0.0666,0.0000,0.0000,0.0000,0.0000,0.0000,1.0379,1.1406,1.5617,1.2260
4,0.1821,0.1613,0.0494,0.0257,0.0884,0.2456,0.0000,0.0000,0.0000,0.0000,0.0000,1.3557,1.4753,2.0829,1.7051
